In [1]:
import pandas as pd
import numpy as np
from pandas import datetime
from matplotlib import pyplot as plt

<ipython-input-1-0c95b2fc7292>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [2]:
"""
Load AirQualityUCI Data
"""

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './data/AirQualityUCI_refined.csv'

df = pd.read_csv(input_file,
                 index_col=[0],
                 parse_dates=[0],
                 date_parser=parser)

df.head()

,CO(GT),PT08.S1(CO),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),RH,AH,C6H6(GT)
Datetime,,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,1046.0,166.0,1056.0,113.0,1692.0,1268.0,48.9,0.7578,11.9
2004-03-10 19:00:00,2.0,1292.0,955.0,103.0,1174.0,92.0,1559.0,972.0,47.7,0.7255,9.4
2004-03-10 20:00:00,2.2,1402.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,54.0,0.7502,9.0
2004-03-10 21:00:00,2.2,1376.0,948.0,172.0,1092.0,122.0,1584.0,1203.0,60.0,0.7867,9.2
2004-03-10 22:00:00,1.6,1272.0,836.0,131.0,1205.0,116.0,1490.0,1110.0,59.6,0.7888,6.5


In [3]:
# Visualization setup
%matplotlib
%config InlineBackend.figure_format = 'svg'

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
plt.ion() # enable the interactive mode

import seaborn as sns
sns.set()  # set plot styles

Using matplotlib backend: Qt5Agg


In [4]:
# Visualize the 'CO(GT)' column
df['CO(GT)'].interpolate(inplace=True) 
# inplace -> dataframe 원본에 덮어쓰기

# 원본 건들기 싫으면
"""
co = df['CO(GT)'].copy()
co.interpolate(inplace=True)
# co객체의 결측치들을 메꾼 다음 그 결과를 그대로 co에 덮어쓰기
"""
# bining 수행하기 위해 interpolation
"""
시계열에서 결측치는 거의 허용되지 않음
만약 일반 설문조사에서 1~10번 중 7번 문항 응답 안 했다(결측데이터)고 해서 뒤에 있을 8,9,10번 문제에 영향을 주는 경우는 거의 없다
시계열이 아닌 데이터는 결측치가 발생해도 영향을 주는 경우는 거의 없다
시계열은 그 자체로 과거 데이터와 미래 데이터 간 의존성을 가지고 있어 중간에 결측치가 생길 경우 무시하기가 어렵다(퍼포먼스 나빠짐)
결측치는 어떻게든 메꿔서 
"""

In [5]:
"""
Binning
"""

# 구간을 나누기 위해 최대,최소값 이용(범위를 몇개의 구간으로 나눌지 정할 수 있다)
max_val = df['CO(GT)'].max()
min_val = df['CO(GT)'].min()
print(max_val, min_val)

11.9 0.0


In [6]:
# Make interval values
# 인터벌 값 max와 min 사이 간격 나누기
bins = np.linspace(min_val, max_val, 6) # 6개 기준점으로 나눈다
bins

array([ 0.  ,  2.38,  4.76,  7.14,  9.52, 11.9 ])

In [7]:
# Labels for each bin
# 5개
labels=['0 <=x<2.38', '2.38<=x<4.76', '4.76<=x<7.14',
       '7.14<=x<9.52', '9.52<=x<11.9']

In [8]:
# Convert the numerical values into the categorical values
df['bins'] = pd.cut(df['CO(GT)'], bins=bins,
                    labels=labels, include_lowest=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9357 entries, 2004-03-10 18:00:00 to 2005-04-04 14:00:00
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   CO(GT)         9357 non-null   float64 
 1   PT08.S1(CO)    8991 non-null   float64 
 2   PT08.S2(NMHC)  8991 non-null   float64 
 3   NOx(GT)        7718 non-null   float64 
 4   PT08.S3(NOx)   8991 non-null   float64 
 5   NO2(GT)        7715 non-null   float64 
 6   PT08.S4(NO2)   8991 non-null   float64 
 7   PT08.S5(O3)    8991 non-null   float64 
 8   RH             8991 non-null   float64 
 9   AH             8991 non-null   float64 
 10  C6H6(GT)       9357 non-null   float64 
 11  bins           9357 non-null   category
dtypes: category(1), float64(11)
memory usage: 886.6 KB


In [9]:
# Print bins
df['bins'][:50]

Datetime
2004-03-10 18:00:00    2.38<=x<4.76
2004-03-10 19:00:00      0 <=x<2.38
2004-03-10 20:00:00      0 <=x<2.38
2004-03-10 21:00:00      0 <=x<2.38
2004-03-10 22:00:00      0 <=x<2.38
2004-03-10 23:00:00      0 <=x<2.38
2004-03-11 00:00:00      0 <=x<2.38
2004-03-11 01:00:00      0 <=x<2.38
2004-03-11 02:00:00      0 <=x<2.38
2004-03-11 03:00:00      0 <=x<2.38
2004-03-11 04:00:00      0 <=x<2.38
2004-03-11 05:00:00      0 <=x<2.38
2004-03-11 06:00:00      0 <=x<2.38
2004-03-11 07:00:00      0 <=x<2.38
2004-03-11 08:00:00      0 <=x<2.38
2004-03-11 09:00:00      0 <=x<2.38
2004-03-11 10:00:00      0 <=x<2.38
2004-03-11 11:00:00      0 <=x<2.38
2004-03-11 12:00:00      0 <=x<2.38
2004-03-11 13:00:00      0 <=x<2.38
2004-03-11 14:00:00    2.38<=x<4.76
2004-03-11 15:00:00      0 <=x<2.38
2004-03-11 16:00:00      0 <=x<2.38
2004-03-11 17:00:00    2.38<=x<4.76
2004-03-11 18:00:00    4.76<=x<7.14
2004-03-11 19:00:00    4.76<=x<7.14
2004-03-11 20:00:00    4.76<=x<7.14
2004-03-11 21:00:00

In [10]:
# Visualize the histogram of bins
plt.hist(df['bins'], bins=5)
plt.show()
# 히스토그램 형태로 데이터 시각화

In [11]:
"""
Log Transform
"""

# Distribution of original data
sns.distplot(df['PT08.S3(NOx)']) # 질소 산화물 변수

# 약간 우편향된 데이터

C:\Users\hgkim\anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<AxesSubplot:xlabel='PT08.S3(NOx)', ylabel='Density'>

In [12]:
# Calculate natural logarithm on 'CO(GT)' column
df['log'] = np.log10(df['PT08.S3(NOx)'])
# np.log 는 자연로그 np.log10은 상용로그

In [13]:
# Min values for each column
df.min()

CO(GT)                  0.0
PT08.S1(CO)           647.0
PT08.S2(NMHC)         383.0
NOx(GT)                 2.0
PT08.S3(NOx)          322.0
NO2(GT)                 2.0
PT08.S4(NO2)          551.0
PT08.S5(O3)           221.0
RH                      9.2
AH                   0.1847
C6H6(GT)                0.0
bins             0 <=x<2.38
log                2.507856
dtype: object

In [14]:
# Distribution after log transform
sns.distplot(df['log'])
plt.xlabel('log(PT08.S3(NoX))')
plt.show()
# 분포 변화

C:\Users\hgkim\anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [16]:
"""
One-hot Encoding
"""

# Make a dataset
# 직원과 관련 (ID, 성별, 평가)
emp_id = pd.Series([1, 2, 3, 4, 5])
gender = pd.Series(['Male', 'Female', 'Female', 'Male', 'Female'])
remarks = pd.Series(['Nice', 'Good', 'Great', 'Great', 'Nice'])
# 각각을 Series로 만든 후

# 빈 DataFrame을 만들어서 Series를 하나씩 coloumn으로 추가
df_emp = pd.DataFrame() 
df_emp['emp_id'] = emp_id
df_emp['gender'] = gender
df_emp['remarks'] = remarks


In [17]:
# Print DataFrame
df_emp

,emp_id,gender,remarks
0,1,Male,Nice
1,2,Female,Good
2,3,Female,Great
3,4,Male,Great
4,5,Female,Nice


In [18]:
# Print unique values for each column
print(df_emp['emp_id'].unique()) # id는 개별적으로 다름 / 예측력을 가지는 변수가 아니라 필요 없어
print(df_emp['gender'].unique()) # 범주 2개 / 2개의 특징으로 구성된 이진벡터
print(df_emp['remarks'].unique()) # 범주 3개 / 3개의 특징으로 구성된 이진벡터
# 해당 coloumn에서 가능한 범주, unique한 값을 찾아 출력해줌

[1 2 3 4 5]
['Male' 'Female']
['Nice' 'Good' 'Great']


In [19]:
# One-hot encoding the categorial values
df_emp_encoded = pd.get_dummies(
    df_emp, columns = ['gender', 'remarks'])

df_emp_encoded
# One-hot encoding이 옛날에는 dummy data였음  (그래서 메소드 명이 get_dummies)
# 원본데이터, coloumns = [coloumn 선택]

,emp_id,gender_Female,gender_Male,remarks_Good,remarks_Great,remarks_Nice
0,1,0,1,0,0,1
1,2,1,0,1,0,0
2,3,1,0,0,1,0
3,4,0,1,0,1,0
4,5,1,0,0,0,1


In [20]:
"""
Normalization
"""

# Visualize two columns of different scales
plt.plot(df['CO(GT)'], label='CO')
plt.plot(df['PT08.S2(NMHC)'], label='NMHC')
plt.legend(loc='best')
# 두 시계열의 스케일 차이가 엄청남

In [21]:
# Normalize the 'CO(GT)' column
co = df['CO(GT)'].copy()
co_max = co.max()
co_min = co.min()
# 최대 최소값 구해서

df['CO_Norm'] = (co - co_min) / (co_max - co_min) # 수식 적용
df['CO_Norm']

Datetime
2004-03-10 18:00:00    0.218487
2004-03-10 19:00:00    0.168067
2004-03-10 20:00:00    0.184874
2004-03-10 21:00:00    0.184874
2004-03-10 22:00:00    0.134454
                         ...   
2005-04-04 10:00:00    0.260504
2005-04-04 11:00:00    0.201681
2005-04-04 12:00:00    0.201681
2005-04-04 13:00:00    0.176471
2005-04-04 14:00:00    0.184874
Name: CO_Norm, Length: 9357, dtype: float64

In [22]:
# Normalize the 'PT08.S2(NMHC)' column
nmhc = df['PT08.S2(NMHC)'].copy()
nmhc_max = nmhc.max()
nmhc_min = nmhc.min()

df['NMHC_Norm'] = (nmhc - nmhc_min) / (nmhc_max - nmhc_min)
df['NMHC_Norm']

# 스케일 조정

Datetime
2004-03-10 18:00:00    0.362097
2004-03-10 19:00:00    0.312398
2004-03-10 20:00:00    0.303659
2004-03-10 21:00:00    0.308575
2004-03-10 22:00:00    0.247406
                         ...   
2005-04-04 10:00:00    0.392135
2005-04-04 11:00:00    0.351720
2005-04-04 12:00:00    0.371382
2005-04-04 13:00:00    0.315674
2005-04-04 14:00:00    0.362643
Name: NMHC_Norm, Length: 9357, dtype: float64

In [23]:
# Visualized normalized columns
plt.plot(df['CO_Norm'], label='CO (normalized)')
plt.plot(df['NMHC_Norm'], label='NMHC (normalized)')
plt.legend(loc='best')
# 양의 상관성이 높은 시계열